_THIS `.qmd` FILE SHOULD BE USED TO WRITE YOUR REPORT.  YOU WILL NEED TO COMPILE THE REPORT INTO A `.html` DOCUMENT AND SUBMIT IT ON CANVAS._


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 
import altair as alt 
import numpy as np 
import altair as alt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import tree

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, RocCurveDisplay

from IPython.display import Markdown
from IPython.display import display
from tabulate import tabulate

## Elevator pitch

_This project uses machine learning techniques to predict whether a dwelling was built before 1980. By analyzing features such as living area, number of bedrooms and bathrooms, and whether the property has a basement, the model is able to accurately predict the age of a dwelling. This type of analysis can be valuable to a range of clients, such as real estate investors, city planners, and housing agencies. It can help them make informed decisions about investing in certain areas, identifying neighborhoods that may be in need of rehabilitation, and ensuring that affordable housing is available to those who need it._


In [ ]:
dwellings_ml = pd.read_csv('dwellings_ml.csv')
dwellings_denver = pd.read_csv('dwellings_denver.csv')
dwellings_neighborhoods_ml = pd.read_csv('dwellings_neighborhoods_ml.csv')

alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('data_server')

## GRAND QUESTION 1

__Create 2-3 charts that evaluate potential relationships between the home variables and before1980.__


In [ ]:
sns.scatterplot(x='finbsmnt', y='basement', hue='before1980', data=dwellings_ml)

In [ ]:
sns.scatterplot(x='livearea', y='sprice', hue='before1980', data=dwellings_ml)

In [ ]:
sns.scatterplot(x='livearea', y='totunits', hue='before1980', data=dwellings_ml)

## GRAND QUESTION 2

__Build a classification model labeling houses as being built “before 1980” or “during or after 1980”. Your goal is to reach or exceed 90% accuracy. Explain your final model choice (algorithm, tuning parameters, etc) and describe what other models you tried.__


In [ ]:
X_pred = dwellings_ml.filter(regex='sprice|livearea|finbsmnt|basement|totunits|stories|numbaths|numbdrm|abstrprd')
y_pred = dwellings_ml.filter(regex = "before1980")
X_train, X_test, y_train, y_test = train_test_split(
X_pred, y_pred, test_size = .34, random_state = 76)  

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_probs = clf.predict_proba(X_test)

## GRAND QUESTION 3

__Justify your classification model by discussing the most important features selected by your model. This discussion should include a chart and a description of the features.__


In [ ]:
print(metrics.classification_report(y_pred, y_test))

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

y_probs = clf.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_probs)


roc_auc = auc(fpr, tpr)


roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='Decision Tree')
roc_display.plot()
plt.show()

## GRAND QUESTION 4

__Describe the quality of your classification model using 2-3 different evaluation metrics. You also need to explain how to interpret each of the evaluation metrics you use.__


In [ ]:
df_features = pd.DataFrame(
    {'f_names': X_train.columns, 
    'f_values': clf.feature_importances_}).sort_values('f_values', ascending = False)

In [ ]:
(alt.Chart(df_features.query('f_values > .011'))
    .encode(
        alt.X('f_values'),
        alt.Y('f_names', sort = '-x'))
    .mark_bar())